In [2]:
import math
import validators
import spacy
from spacy import displacy
import pandas as pd
import numpy as np
import os
import email
import email.policy
from bs4 import BeautifulSoup
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.tokenize import RegexpTokenizer,word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import word2vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import re
import heapq
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.feature_extraction.text import HashingVectorizer

## CARREGANDO BASE DE DADOS

In [3]:
database = pd.read_csv("Database/enron_spam_data.csv")
database = database.drop(columns=["Message ID","Date"])
database = database.replace({"ham":0,"spam":1})
database = database.drop(columns=["Subject"])
database = database.dropna()
database = database.sample(frac=1)
target = database["Spam/Ham"]

In [5]:
def removePunctuation(text):
    ponctuation = list(punctuation)

    for i in ponctuation:
        text = text.replace(i, "")

    return text

stopWords = set(stopwords.words('english')  + list(punctuation) + list(STOPWORDS))
stopWords.add("subject")
stem = PorterStemmer()
lemmatizer = WordNetLemmatizer()

regex = re.compile(r'[\w\.-]+@[\w\.-]+(\.[\w]+)+')

def wordsPreProcessing(email):
    if email is None:
        return 'empty'

    newText = ""

    e = email.split()

    for i in range(0,len(e)):
        if validators.url(e[i]):
            e[i] = "URLLL"

        if re.fullmatch(regex,e[i]):
            e[i] = "EMAILLL"

    for text in word_tokenize(email.lower()):
        text = removePunctuation(text)
        if text not in stopWords and not text.isdigit():
            newText += lemmatizer.lemmatize(text) + " "

    return newText

def NERProcessing(emails):
    emailsPreProcessedLocal = []

    spacy.require_gpu()
    NER = spacy.load("en_core_web_sm")
    NER.disable_pipe("parser")
    NER.add_pipe("sentencizer")
    NER.add_pipe("merge_entities")

    for doc in NER.pipe(emails, n_process=2):
        emailsPreProcessedAux = []

        for t in doc:
            if not t.ent_type_:
                emailsPreProcessedAux.append(t.text)
            else:
                emailsPreProcessedAux.append((t.ent_type_ + t.ent_type_[-1]).upper())

        emailsPreProcessedAux = " ".join(emailsPreProcessedAux)
        emailsPreProcessedLocal.append(wordsPreProcessing(emailsPreProcessedAux))

    return emailsPreProcessedLocal

In [9]:
textos = []
ini = datetime.now()

emailsPreProcessed = NERProcessing(database["Message"])

print(datetime.now() - ini)

In [ ]:
NERDataframe = pd.DataFrame(emailsPreProcessed, columns=["email"])
NERDataframe["target"] = target.values

NERDataframe.to_csv('Database/dataBaseWithNER.csv')
NERDataframe